# Marginals, Haar

In [1]:
import sys
sys.path.insert(0, '../../src/')

import numpy as np
from analysis import hist_ensamble
from statsmodels.nonparametric.kernel_regression import KernelReg

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


In [2]:
#pandas load .dat
import pandas as pd
import os
import numpy as np

def load_data(file_path):
    # Check if the file exists
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"The file {file_path} does not exist.")

    # Load the data using pandas
    data = pd.read_csv(file_path, delim_whitespace=True, header=None)
    
    # Convert the DataFrame to a NumPy array
    data_array = data.to_numpy()
    
    return data_array

import re
import numpy as np

# pre-compile the regex for performance
_exp_pat = re.compile(r'\*\^([+-]?\d+)')


def to_complex_array(obj_arr: np.ndarray) -> np.ndarray:
    """
    Convert an array of strings of the form
      - 'a+bi*I'
      - 'a-bi*I'
      - 'a*^b+ci*I'
      - 'a*^b-ci*I'
    into a numpy array of complex numbers.
    
    Parameters
    ----------
    obj_arr : np.ndarray
        Object-dtype (or string-dtype) array where each entry is a string
        like '0.5+1.2*I' or '-3.9*^-7-0.37*I'.
    
    Returns
    -------
    np.ndarray
        Array of dtype complex128 with the same shape as obj_arr.
    """
    str_arr = obj_arr.astype(str)
    
    def _convert_one(s: str) -> str:
        # 1) replace "*^b" → "eb"
        s = _exp_pat.sub(r'e\1', s)
        # 2) replace "*I" → "j"
        s = s.replace('*I', 'j')
        return s
    
    # vectorize across the array
    vect = np.vectorize(_convert_one, otypes=[str])
    j_arr = vect(str_arr)
    
    # finally cast to complex128
    return j_arr.astype(np.complex128)

In [15]:
raw = load_data("data/AI_4_1.large")
AI_4_1 = to_complex_array(raw)

C:\Users\krist\AppData\Local\Temp\ipykernel_1512\3382094422.py:12: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(file_path, delim_whitespace=True, header=None)


## Radial and Angular

### AI

In [16]:
radial = np.abs(AI_4_1)
angular = np.angle(AI_4_1)

print(np.mean(radial))
print(-np.mean(np.cos(angular)))

r_bins = np.linspace(0, 1, 100)
r_bins_large = np.linspace(0, 1, 1000)
r_cbin_large = (r_bins_large[:-1] + r_bins_large[1:]) / 2

a_bins = np.linspace(-np.pi, np.pi, 100)
a_bins_large = np.linspace(-np.pi, np.pi, 1000)
a_cbin_large = (a_bins_large[:-1] + a_bins_large[1:]) / 2

center_bins_r, hist_r, _ = hist_ensamble([radial], r_bins)
center_bins_a, hist_a, _ = hist_ensamble([angular], a_bins)

kr = KernelReg(endog=hist_r, exog=center_bins_r, var_type='c', bw=[0.05]) 
hist_r, _ = kr.fit(r_cbin_large.reshape(-1, 1))

kr = KernelReg(endog=hist_a, exog=center_bins_a, var_type='c', bw=[0.05]) 
hist_a, _ = kr.fit(a_cbin_large.reshape(-1, 1))


radial_AI_hist = np.array([r_cbin_large, hist_r]).T
angular_AI_hist = np.array([a_cbin_large, hist_a]).T

np.savetxt('bookkeeping/figure3/radial_AI_theory.txt', radial_AI_hist)
np.savetxt('bookkeeping/figure3/angular_AI_theory.txt', angular_AI_hist)

0.730666908336575
0.19849352312582436


### FF

In [17]:
raw = load_data("data/FF_4_1_nosel_amend.large")
FF_4_1 = to_complex_array(raw)

C:\Users\krist\AppData\Local\Temp\ipykernel_1512\3382094422.py:12: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(file_path, delim_whitespace=True, header=None)


In [19]:
radial = np.abs(FF_4_1)
angular = np.angle(FF_4_1)

print(np.mean(radial))
print(-np.mean(np.cos(angular)))

r_bins = np.linspace(0, 1, 100)
r_bins_large = np.linspace(0, 1, 1000)
r_cbin_large = (r_bins_large[:-1] + r_bins_large[1:]) / 2

a_bins = np.linspace(-np.pi, np.pi, 100)
a_bins_large = np.linspace(-np.pi, np.pi, 1000)
a_cbin_large = (a_bins_large[:-1] + a_bins_large[1:]) / 2

center_bins_r, hist_r, _ = hist_ensamble([radial], r_bins)
center_bins_a, hist_a, _ = hist_ensamble([angular], a_bins)

kr = KernelReg(endog=hist_r, exog=center_bins_r, var_type='c', bw=[0.05]) 
hist_r, _ = kr.fit(r_cbin_large.reshape(-1, 1))

kr = KernelReg(endog=hist_a, exog=center_bins_a, var_type='c', bw=[0.05]) 
hist_a, _ = kr.fit(a_cbin_large.reshape(-1, 1))

print(hist_r.shape)

radial_FF_hist = np.array([r_cbin_large, hist_r]).T
angular_FF_hist = np.array([a_cbin_large, hist_a]).T

np.savetxt('bookkeeping/figure3/radial_FF_theory.txt', radial_FF_hist)
np.savetxt('bookkeeping/figure3/angular_FF_theory.txt', angular_FF_hist)

0.6177348961486886
-0.09610440656318214
(999,)
